In [11]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from unicodedata import normalize
import string
import time
import datetime

In [12]:
df = pd.read_csv('Player_Stats_Reg3.csv')

In [18]:
df.shape

(45790, 29)

In [15]:
def get_pstats(year):


    url = 'https://www.hockey-reference.com/leagues/NHL_'+year+'_skaters.html'
    dfsi = pd.read_html(url)
    dfi = dfsi[0]
    dfi = dfi.loc[dfi['Unnamed: 0_level_0']['Rk'] != 'Rk']
    dfi.columns = dfi.columns.droplevel()
    #scraping initial dataframe from hockey-reference.com

    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    slinks = soup.select("th+ .left a")
    surl = [s['href'] for s in slinks]
    #getting list of playerids from links

    pid = [surl[i].split('/')[3].split('.')[0] for i in range(len(surl))]  #adding playerid column
    dfi['pid'] = pid
    dfi = dfi.loc[dfi['Tm'] != 'TOT']  #dropping total rows
    dfi['Year'] = str(int(year)-1)+ '-' + year             #adding year column

    dfi['Player'] = dfi['Player'].str.rstrip('*') #removing * symbol from hall of fame players' names
    dfi['YR_INT'] = dfi['Year'].str[-4:]          #adding col for year as integer
#     dfi['YR_INT'] = dfi['YR_INT'].astype(int)
    dfi = dfi.drop(columns = ['Rk'])
    dfi = dfi.drop(columns = ['ATOI'])            #drop average time on ice column for sql
    dfi.columns = ['Player', 'Age', 'Tm', 'Pos', 'GP', 'G', 'A', 'PTS', '+/-', 'PIM', 'PS', 'EVG', 'PPG', 'SHG', 'GWG', 
     'EVA', 'PPA', 'SHA', 'S', 'S%', 'TOI', 'BLK', 'HIT', 'FOW', 'FOL', 'FO%', 'pid', 'Year', 'YR_INT']  #changing column names -more descriptive
    
    
    
#     url_string = 'backup/csv/'+str(year)+'pstats_'+datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d')+'.csv'
#     dfi.to_csv(url_string, index = False)
    
    print('Updated Season Stats:')
    print(dfi.head())
    print(dfi.shape)
#     print(f'writing to {url_string}')
    print('\n')
    return dfi

dfi = get_pstats('2021')
dfi

Updated Season Stats:
            Player Age   Tm Pos  GP  G  A PTS +/- PIM  ...   S%  TOI BLK HIT  \
0   Vitaly Abramov  22  OTT  RW   2  0  0   0   0   0  ...  NaN   19   0   0   
1     Noel Acciari  29  FLA   C  41  4  7  11   0   9  ...  5.9  631  63  91   
2    Calen Addison  20  MIN   D   3  0  0   0   0   0  ...  0.0   56   1   2   
3   Kenny Agostino  28  TOR  LW   1  0  0   0   0   0  ...  NaN    4   0   0   
4  Andrew Agozzino  30  ANA  LW   3  0  1   1   0   0  ...  0.0   37   7   6   

   FOW  FOL   FO%        pid       Year YR_INT  
0    0    0   NaN  abramvi01  2020-2021   2021  
1  272  247  52.4  acciano01  2020-2021   2021  
2    0    0   NaN  addisca01  2020-2021   2021  
3    0    0   NaN  agostke01  2020-2021   2021  
4   16   16  50.0  agozzan01  2020-2021   2021  

[5 rows x 29 columns]
(972, 29)




<ipython-input-15-c49f0033bfb7>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfi['pid'] = pid


,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,PIM,...,S%,TOI,BLK,HIT,FOW,FOL,FO%,pid,Year,YR_INT
0,Vitaly Abramov,22,OTT,RW,2,0,0,0,0,0,...,NaN,19,0,0,0,0,NaN,abramvi01,2020-2021,2021
1,Noel Acciari,29,FLA,C,41,4,7,11,0,9,...,5.9,631,63,91,272,247,52.4,acciano01,2020-2021,2021
2,Calen Addison,20,MIN,D,3,0,0,0,0,0,...,0.0,56,1,2,0,0,NaN,addisca01,2020-2021,2021
3,Kenny Agostino,28,TOR,LW,1,0,0,0,0,0,...,NaN,4,0,0,0,0,NaN,agostke01,2020-2021,2021
4,Andrew Agozzino,30,ANA,LW,3,0,1,1,0,0,...,0.0,37,7,6,16,16,50.0,agozzan01,2020-2021,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,Mika Zibanejad,27,NYR,C,56,24,26,50,2,18,...,14.0,1126,44,75,479,555,46.3,zibanmi01,2020-2021,2021
1071,Radim Zohorna,24,PIT,F,8,2,2,4,4,4,...,25.0,72,0,7,5,3,62.5,zohorra01,2020-2021,2021
1072,Artem Zub,25,OTT,D,47,3,11,14,4,26,...,7.7,864,70,89,0,0,NaN,zubar01,2020-2021,2021
1073,Mats Zuccarello,33,MIN,LW,42,11,24,35,9,8,...,14.9,701,19,12,11,21,34.4,zuccama01,2020-2021,2021


In [19]:
dfc = pd.concat([df, dfi])
dfc.to_csv('Player_Stats_Reg.csv', index = False)

In [20]:
dfg = pd.read_csv('Goalie_Stats_Reg.csv')
dfg

,Player,Age,Tm,GP,GS,W,L,T/O,GA,SA,...,RBS,GA%-,GSAA,G,A,PTS,PIM,pid,Year,YR_INT
0,Jake Allen,29,STL,24,21.0,12.0,6.0,3.0,48,655.0,...,2.0,81.0,11.23,0,0.0,0.0,0,allenja01,2019-2020,2020
1,Frederik Andersen,30,TOR,52,52.0,29.0,13.0,7.0,143,1577.0,...,8.0,100.0,-0.40,0,0.0,0.0,0,anderfr01,2019-2020,2020
2,Craig Anderson,38,OTT,34,31.0,11.0,17.0,2.0,100,1018.0,...,7.0,109.0,-7.95,0,1.0,1.0,0,andercr01,2019-2020,2020
3,David Ayres,42,CAR,1,0.0,1.0,0.0,0.0,2,10.0,...,0.0,NaN,NaN,0,0.0,0.0,0,ayresda01,2019-2020,2020
4,Jonathan Bernier,31,DET,46,41.0,15.0,22.0,3.0,126,1353.0,...,5.0,103.0,-3.66,0,3.0,3.0,4,bernijo01,2019-2020,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4612,Art Brooks,28,TRA,4,NaN,2.0,2.0,0.0,23,NaN,...,NaN,NaN,NaN,0,0.0,0.0,0,brookar01,1917-1918,1918
4613,Sammy Hebert,23,TRA,2,NaN,1.0,0.0,0.0,10,NaN,...,NaN,NaN,NaN,0,0.0,0.0,0,hebersa01,1917-1918,1918
4614,Hap Holmes,29,TRA,16,NaN,9.0,7.0,0.0,76,NaN,...,NaN,NaN,NaN,0,0.0,0.0,0,holmeha01,1917-1918,1918
4615,Bert Lindsay,36,MTW,4,NaN,1.0,3.0,0.0,35,NaN,...,NaN,NaN,NaN,0,0.0,0.0,0,lindsbe01,1917-1918,1918


In [24]:

i = 2021
url = 'https://www.hockey-reference.com/leagues/NHL_'+str(i)+'_goalies.html'
dfsi = pd.read_html(url)
dfi = dfsi[0]

dfi.columns = dfi.columns.droplevel()
dfi = dfi.loc[dfi['Rk'] != 'Rk']


resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')
slinks = soup.select("th+ .left a")
surl = [s['href'] for s in slinks]
#getting list of playerids from links

pid = [surl[i].split('/')[3].split('.')[0] for i in range(len(surl))]  #adding playerid column
dfi['pid'] = pid

dfi = dfi.loc[dfi['Tm'] != 'TOT']  #dropping total rows

py = i -1    
year_col_title = str(py)+'-'+str(i)
dfi['Year'] = year_col_title

print(i, dfi.shape)

df = dfi


        
df['Player'] = df['Player'].str.rstrip('*') #removing * symbol from hall of fame players' names
df['YR_INT'] = df['Year'].str[-4:]          #adding col for year as integer
df['YR_INT'] = df['YR_INT'].astype(int)
df = df.drop(columns = ['Rk'])
df

2021 (101, 28)


<ipython-input-24-c7f32127dc5e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfi['pid'] = pid


,Player,Age,Tm,GP,GS,W,L,T/O,GA,SA,...,RBS,GA%-,GSAA,G,A,PTS,PIM,pid,Year,YR_INT
0,Jake Allen,30,MTL,29,27,11,12,5,76,817,...,2,101,-0.73,0,0,0,2,allenja01,2020-2021,2021
1,Frederik Andersen,31,TOR,24,23,13,8,3,70,668,...,6,114,-8.46,0,1,1,0,anderfr01,2020-2021,2021
2,Craig Anderson,39,WSH,4,2,2,1,0,6,71,...,0,NaN,NaN,0,0,0,0,andercr01,2020-2021,2021
3,Jonathan Bernier,32,DET,24,23,9,11,1,65,757,...,4,93,4.74,0,0,0,0,bernijo01,2020-2021,2021
4,Jordan Binnington,27,STL,42,41,18,14,8,108,1200,...,5,98,2.56,0,1,1,2,binnijo01,2020-2021,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Andrei Vasilevskiy,26,TBL,42,42,31,10,1,93,1237,...,0,82,20.96,0,4,4,0,vasilan02,2020-2021,2021
104,Veini Vehvilainen,23,CBJ,1,0,0,0,0,1,4,...,0,NaN,NaN,0,0,0,0,vehvive01,2020-2021,2021
105,Daniel Vladar,23,BOS,5,5,2,2,1,17,149,...,1,NaN,NaN,0,0,0,0,vladada01,2020-2021,2021
106,Scott Wedgewood,28,NJD,16,15,3,8,3,46,458,...,4,109,-3.80,0,0,0,0,wedgesc01,2020-2021,2021


In [25]:
dfgc = pd.concat([dfg, df])
dfgc

,Player,Age,Tm,GP,GS,W,L,T/O,GA,SA,...,RBS,GA%-,GSAA,G,A,PTS,PIM,pid,Year,YR_INT
0,Jake Allen,29,STL,24,21,12,6,3,48,655,...,2,81,11.23,0,0,0,0,allenja01,2019-2020,2020
1,Frederik Andersen,30,TOR,52,52,29,13,7,143,1577,...,8,100,-0.4,0,0,0,0,anderfr01,2019-2020,2020
2,Craig Anderson,38,OTT,34,31,11,17,2,100,1018,...,7,109,-7.95,0,1,1,0,andercr01,2019-2020,2020
3,David Ayres,42,CAR,1,0,1,0,0,2,10,...,0,NaN,NaN,0,0,0,0,ayresda01,2019-2020,2020
4,Jonathan Bernier,31,DET,46,41,15,22,3,126,1353,...,5,103,-3.66,0,3,3,4,bernijo01,2019-2020,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Andrei Vasilevskiy,26,TBL,42,42,31,10,1,93,1237,...,0,82,20.96,0,4,4,0,vasilan02,2020-2021,2021
104,Veini Vehvilainen,23,CBJ,1,0,0,0,0,1,4,...,0,NaN,NaN,0,0,0,0,vehvive01,2020-2021,2021
105,Daniel Vladar,23,BOS,5,5,2,2,1,17,149,...,1,NaN,NaN,0,0,0,0,vladada01,2020-2021,2021
106,Scott Wedgewood,28,NJD,16,15,3,8,3,46,458,...,4,109,-3.80,0,0,0,0,wedgesc01,2020-2021,2021


In [26]:
dfgc.to_csv('Goalie_Stats_Reg.csv', index = False)